## chain

自动组合不同的 LLM 调用和操作

例如：摘要#1、摘要#2、摘要#3、最终摘要


In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

### 1. 简单的顺序链

In [1]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=1)

In [2]:
template = """
你的工作是根据用户建议的区域制作一道经典的菜肴。

% 用户位置
{user_location}

AI 回答:
"""

prompt_template = PromptTemplate(input_variables=["user_location"], template=template)

# 查询某地的一个菜谱（针对llm进行查询的链）
location_chain = LLMChain(llm=llm, prompt=prompt_template)

In [3]:
template = """
给出一个简短的食谱，说明如何在家做这道菜。

% 菜谱
{user_meal}

AI 回答:
"""

prompt_template = PromptTemplate(input_variables=["user_meal"], template=template)

# 查询某个菜谱的制作方法
meal_chain = LLMChain(llm=llm, prompt=prompt_template)

In [4]:
# 汇总链
overall_chain = SimpleSequentialChain(chains=[location_chain, meal_chain], verbose=True)

review = overall_chain.run("河南南阳")



> Entering new SimpleSequentialChain chain...
首先，河南南阳有很多经典的菜肴，包括河南南阳炒鸡、汤圆饼、莲藕粉和蒸面条。我们可以用当地的食材来制作这些经典菜肴，例如将鸡胸肉、葱、姜、白胡椒和豆腐片炒做成河南南阳炒鸡；把豆沙、红薯和玉米糊混合制作成汤圆饼；将莲藕

1. 准备材料: 鸡胸肉、葱、姜、白胡椒、豆腐片、豆沙、红薯、玉米糊。
2. 将鸡胸肉切成小块，加入葱、姜、白胡椒，然后用油炒至金黄色。最后放入豆腐片，再炒透。
3. 将豆沙、红薯、玉米糊混合，制作成汤圆饼。
4. 将莲藕洗净，然后切成薄片。
5

> Finished chain.


### 2. 总结链

In [12]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader('data/llama2.txt', encoding='utf-8')
documents = loader.load()

# 准备好分离器
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=50)

# 将文档拆分为文本
texts = text_splitter.split_documents(documents)


chain = load_summarize_chain(llm, chain_type='map_reduce', verbose=True)
chain.run(texts)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Llama 2 is here - get it on Hugging Face"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"Introduction
Llama 2 is a family of state-of-the-art open-access large language models released by Meta today, and we’re excited to fully support the launch with comprehensive integration in Hugging Face. Llama 2 is being released with a very permissive community license and is available for commercial use. The code, pretrained models, and fine-tuned models are all being released today 🔥"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"We’ve collaborated with Meta to ensure smooth integration into the Hugging Face ecosystem. You can find the 12 open-access models (3 base models & 3 fine-tuned ones with the original Meta checkpoints, plus their corresponding transfo


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"
Llama 2 has been released and is available to download on Hugging Face.


Meta has released "Llama 2", a family of powerful open-access large language models, to the public with a very permissive license and full integration into Hugging Face. All associated code, pretrained models, and fine-tuned models are also available today.

 Hugging Face has collaborated with Meta to create twelve available open-access models, now found on their Hub. These models have been integrated into the Hugging Face ecosystem, along with features such as model cards and licenses, Transformers integration, fine-tuning examples, text generation inference, and inference endpoints.

 This Table of Contents covers the release of Llama 2, a family of pretrained and fine-tuned LLMs ranging from 7B to 70B parameters, which provides significant improvements compared to Llama 1, such as 40% mo

" Meta has released Llama 2, a family of powerful large language models, available with a very permissive license. This includes twelve open-access models released to Hugging Face's Hub, with associated features such as model cards and licenses, Transformers integration, fine-tuning, and inference endpoints. Additionally, LLM2-Chat, a Reinforcement Learning chatbot, is now available as the best open-sourced alternative. A code snippet provides instructions to generate a recommended TV show from a given input. Text-Generation Inference is a container for LLM2 support with 4K context size, hyperparameters, and streaming APIs; and users can use a fine-tuning script with tools from the Hugging Face ecosystem. This code also includes a template for conversations with a system prompt that should not include any offensive content and follow the <s> structure."